In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET 

In [2]:
def breaker() -> None:
    print("\n" + 50*"*" + "\n")


def get_size(ele) -> tuple:
    return int(ele[0].text), int(ele[1].text), int(ele[2].text)


def get_bbox(ele, w: int, h: int) -> tuple:
    return int(ele[0].text)/w, int(ele[1].text)/h, int(ele[2].text)/w, int(ele[3].text)/h


def get_image(path: str, size: int=224) -> np.ndarray:
    return cv2.resize(src=cv2.cvtColor(src=cv2.imread(path, cv2.IMREAD_COLOR), code=cv2.COLOR_BGR2RGB), dsize=(size, size), interpolation=cv2.INTER_AREA)


def get_images(base_path: str, filenames: np.ndarray, size: int=224) -> np.ndarray:
    images = np.zeros((len(filenames), size, size, 3), dtype=np.uint8)
    
    i = 0
    for filename in filenames:
        images[i] = get_image(os.path.join(base_path, filename), size=size)
        i += 1
    
    return images


def get_statistics(images: np.ndarray) -> tuple:
    return (images[:, :, :, 0].mean()/255, images[:, :, :, 1].mean()/255, images[:, :, :, 2].mean()/255), (images[:, :, :, 0].std()/255, images[:, :, :, 1].std()/255, images[:, :, :, 2].std()/255)

In [3]:
XML_FILES_PATH = "../input/car-plate-detection/annotations"

xml_filenames = sorted(os.listdir(XML_FILES_PATH))
filenames = [filename[:-3]+"png" for filename in xml_filenames]

x1s, y1s, x2s, y2s = [], [], [], []

for xml_file in xml_filenames:
    tree = ET.parse(os.path.join(XML_FILES_PATH, xml_file))
    root = tree.getroot()
    w, h, c = get_size(root[2])
    (x1, y1, x2, y2) = get_bbox(root[-1][-1], w, h)
    x1s.append(x1)
    y1s.append(y1)
    x2s.append(x2)
    y2s.append(y2)

In [4]:
df = pd.DataFrame(data=None, columns=["filenames", "x1", "y1", "x2", "y2"])

# bbox = np.concatenate((np.array(x1s).reshape(-1, 1),
#                        np.array(y1s).reshape(-1, 1),
#                        np.array(x2s).reshape(-1, 1),
#                        np.array(y2s).reshape(-1, 1)),
#                      axis=1)

df.filenames = filenames
df.x1 = x1s
df.y1 = y1s
df.x2 = x2s
df.y2 = y2s

df.to_csv("data.csv", index=False)

In [5]:
sizes = [224, 256, 384, 512]
stats = dict()
means = dict()
stds  = dict()

for size in sizes:
    images = get_images("../input/car-plate-detection/images", df.filenames, size)
    mean, std = get_statistics(images)
    means[f"{size}"] = mean
    stds[f"{size}"]  = std
    np.save(f"images_{size}.npy", images)

stats["means"] = means
stats["stds"] = stds

In [6]:
breaker()
for k, v in stats["means"].items():
    print(f"{k} x {k} Means\n")
    for i in range(len(v)):
        if i == 0:
            print(f"Red   : {v[i]:.5f}")
        if i == 1:
            print(f"Green : {v[i]:.5f}")
        if i == 2:
            print(f"Blue  : {v[i]:.5f}")
    breaker()


**************************************************

224 x 224 Means

Red   : 0.42832
Green : 0.42040
Blue  : 0.41810

**************************************************

256 x 256 Means

Red   : 0.42823
Green : 0.42031
Blue  : 0.41801

**************************************************

384 x 384 Means

Red   : 0.42805
Green : 0.42013
Blue  : 0.41784

**************************************************

512 x 512 Means

Red   : 0.42810
Green : 0.42018
Blue  : 0.41788

**************************************************



In [7]:
breaker()
for k, v in stats["stds"].items():
    print(f"{k} x {k} Stds\n")
    for i in range(len(v)):
        if i == 0:
            print(f"Red   : {v[i]:.5f}")
        if i == 1:
            print(f"Green : {v[i]:.5f}")
        if i == 2:
            print(f"Blue  : {v[i]:.5f}")
    breaker()


**************************************************

224 x 224 Stds

Red   : 0.28149
Green : 0.27978
Blue  : 0.28646

**************************************************

256 x 256 Stds

Red   : 0.28222
Green : 0.28052
Blue  : 0.28715

**************************************************

384 x 384 Stds

Red   : 0.28346
Green : 0.28176
Blue  : 0.28835

**************************************************

512 x 512 Stds

Red   : 0.28410
Green : 0.28240
Blue  : 0.28896

**************************************************

